This Jupyter notebook contains the functions needed to run the thresholding algorithm

External packages: pulp, numpy, pandas, networkx, ghudi, matplotlib

In [7]:
# imports, declare necessary PATH variables

path_to_gurobi = r'C:/gurobi1200/win64' # Set to gurobi path in environment variables
import pulp as pl
import numpy as np
import pandas as pd
from gudhi.representations.preprocessing import BirthPersistenceTransform
from gudhi.representations.vector_methods import PersistenceImage
import re

In [10]:
def get_lifetimes(hxdx):
  """
  Generate an array of lifetimes as an nx2 array, where for each entry the first value is the birth and the second value is the death

  hxdx = (pandas DataFrame) the homology information (we use that information generated by FactoredBoundaryMatrixVr's (oat suite) homology method)
          Necessary columns: "dimension", "birth", "death"
          Note: x in hxdx acts a placeholder for specific values, i.e. h1d2 (homology of diagram 1 dimension 2)
  """
  hxdx_list = np.column_stack((hxdx.birth.values.tolist(), hxdx.death.values.tolist()))
  hxdx_list[:,1][hxdx_list[:,1] == np.inf] = 1.00001 # Replace infinite values with numerical value exceeding the filtration maximum

  return hxdx_list



def get_euclidean_distance(homology1_path, homology2_path):
    """
    Calculate the euclidean distance between two persistence diagrams by utilizing their vector image representations

    homology1_path = (String) path to the first homology csv file
    homology2_path = (String) path to the second homology csv file
    """

    # the user can internally add more/different weighting functions

    def linear_weight(arr):
        """
        Weight function weighting persistence image linearly along the y-axis
        
        arr = (np.array) 1x2 array entry of birth persistence transformed persistence diagram
        """
        if type(arr) != type(np.array([])):
            raise TypeError("arr must have type np.array([])")
        return arr[1]
    
    bpt = BirthPersistenceTransform()
    pi = PersistenceImage(bandwidth=0.1, resolution=[20, 20], weight=linear_weight) # the user may also adjust the bandwidth and resolution of the persistence image
    h1 = pd.read_csv(homology1_path)
    h2 = pd.read_csv(homology2_path)

    # Note that e.g. h1d1 refers to the first array's dimension 1 information
    h1d1_lifetimes = get_lifetimes(h1.loc[(h1['dimension'] == 1)])
    h1d2_lifetimes = get_lifetimes(h1.loc[(h1['dimension'] == 2)])
    h2d1_lifetimes = get_lifetimes(h2.loc[(h2['dimension'] == 1)])
    h2d2_lifetimes = get_lifetimes(h2.loc[(h2['dimension'] == 2)])

    if np.any(h1d1_lifetimes):
        h1d1_transformed = bpt(h1d1_lifetimes)
    else:
        h1d1_transformed = np.array([[1.0,1.0]]) # Avoid IndexError (empty array)
    if np.any(h2d1_lifetimes):
        h2d1_transformed = bpt(h2d1_lifetimes)
    else:
        h2d1_transformed = np.array([[1.0,1.0]]) # Avoid IndexError (empty array)
    if np.any(h1d2_lifetimes):
        h1d2_transformed = bpt(h1d2_lifetimes)
    else:
        h1d2_transformed = np.array([[1.0,1.0]]) # Avoid IndexError (empty array)
    if np.any(h2d2_lifetimes):
        h2d2_transformed = bpt(h2d2_lifetimes)
    else:
        h2d2_transformed = np.array([[1.0,1.0]]) # Avoid IndexError (empty array)
    
    h1d1_img = pi(h1d1_transformed)
    h2d1_img = pi(h2d1_transformed)
    h1d2_img = pi(h1d2_transformed)
    h2d2_img = pi(h2d2_transformed)
    
    h1dx_img = np.concatenate((h1d1_img, h1d2_img))
    h2dx_img = np.concatenate((h2d1_img, h2d2_img))
    
    # Euclidean (p=2) distance is calculated using numpy
    temp = np.subtract(h1dx_img, h2dx_img)
    return np.sqrt(np.dot(temp.T, temp))



def networkSize(path):
    """
    Return the number of nodes (D0 features) in a network

    path = (String) path to the homology csv file
    """
    h = pd.read_csv(path)
    return(h.loc[(h['dimension'] == 0)].shape[0])



def featureCountD1(path):
    """
    Filter a homology csv file by dimension, returning where rows are dimension 1

    path = (String) path to the homology csv file
    """
    h = pd.read_csv(path)
    return(h.loc[(h['dimension'] == 1)].shape[0])



def featureCountD2(path):
    """
    Filter a homology csv file by dimension, returning where rows are dimension 2

    path = (String) path to the homology csv file
    """
    h = pd.read_csv(path)
    return(h.loc[(h['dimension'] == 2)].shape[0])



def fd_gridSearch(folder_path, u, l, normalize=False):
    """
    Generate the average weights and feature counts for each file in the specified folder

    folder_path = (String) the folder containing the homology csv files
    u = (float) upper bounds vector
    l = (float) lower bounds vector
    normalize = (Boolean) indicator for whether the feature distrubutions should be normalized by the size of the corresponding network
            Note: the feature counts can normalized by the size of the network. This accounts for bias toward larger networks which tend to comprise more higher-dimensional features inherently
    """
    
    n_i = len(u) # the code is written intended for the indicial convention (i,j) where i is an upper bound, j is a lower bound. The current data reverses this convention
    n_j = len(l)

    gradient_dict = {} # in the gradient dictionary, we map elements of the feature space to the approximated magnitude of the gradient on the latent space manifold located at the vectorized feature

    # Store separated feature counts
    D1_feature_counts = {}
    D2_feature_counts = {}

    for i in range(n_i):
        for j in range(1,n_j): # the data used here includes a lower bound of index 0 which is not considered in the final analysis (i.e. we do not wish to include homology dataframes of the form (0, i).csv); hence it is excluded in the loop

            if normalize:
                D1_feature_counts[(i,j)] = featureCountD1(folder_path + str((j,i)) + ".csv")
                D2_feature_counts[(i,j)] = featureCountD2(folder_path + str((j,i)) + ".csv")
            else:
                D1_feature_counts[(i,j)] = featureCountD1(folder_path + str((j,i)) + ".csv") / networkSize(folder_path + str((j,i)) + ".csv") # Normalize, D1
                D2_feature_counts[(i,j)] = featureCountD2(folder_path + str((j,i)) + ".csv") / networkSize(folder_path + str((j,i)) + ".csv") # Normalize, D2


            if i == 0:
                if j == 0:

                    df_ij_l = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    df_ij_u = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    dl_ij = np.abs(l[j+1] - l[j])
                    du_ij = np.abs(u[i+1] - u[i])

                    gradient_dict[(i,j)] = np.array([df_ij_l / dl_ij, df_ij_u / du_ij])

                elif j == (n_j - 1):

                    df_ij_l = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j-1,i)) + ".csv")
                    df_ij_u = get_euclidean_distance(folder_path + str((j,i+1)) + ".csv", folder_path + str((j,i)) + ".csv")
                    dl_ij = np.abs(l[j] - l[j-1])
                    du_ij = np.abs(u[i+1] - u[i])

                    gradient_dict[(i,j)] = np.array([df_ij_l / dl_ij, df_ij_u / du_ij])

                else:

                    df_ij_l_downstream = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j-1,i)) + ".csv")
                    df_ij_l_upstream = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    df_ij_u = get_euclidean_distance(folder_path + str((j,i+1)) + ".csv", folder_path + str((j,i)) + ".csv")
                    dl_ij_downstream = np.abs(l[j] - l[j-1])
                    dl_ij_upstream = np.abs(l[j+1] - l[j])
                    du_ij = np.abs(u[i+1] - u[i])

                    gradient_dict[(i,j)] = np.array([df_ij_l_downstream / (2*dl_ij_downstream) + df_ij_l_upstream / (2*dl_ij_upstream), df_ij_u / du_ij])

            if i == (n_i - 1):

                if j == 0:

                    df_ij_l = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    df_ij_u = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j,i-1)) + ".csv")
                    dl_ij = np.abs(l[j+1] - l[j])
                    du_ij = np.abs(u[i] - u[i-1])

                    gradient_dict[(i,j)] = np.array([df_ij_l / dl_ij, df_ij_u / du_ij])

                elif j == (n_j - 1):

                    df_ij_l = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j-1,i)) + ".csv")
                    df_ij_u = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j,i-1)) + ".csv")
                    dl_ij = np.abs(l[j] - l[j-1])
                    du_ij = np.abs(u[i] - u[i-1])

                    gradient_dict[(i,j)] = np.array([df_ij_l / dl_ij, df_ij_u / du_ij])

                else:

                    df_ij_l_downstream = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j-1,i)) + ".csv")
                    df_ij_l_upstream = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    df_ij_u = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j,i-1)) + ".csv")
                    dl_ij_downstream = np.abs(l[j] - l[j-1])
                    dl_ij_upstream = np.abs(l[j+1]-l[j])
                    du_ij = np.abs(u[i] - u[i-1])

                    gradient_dict[(i,j)] = np.array([df_ij_l_downstream / (2*dl_ij_downstream) + df_ij_l_upstream / (2*dl_ij_upstream), df_ij_u / du_ij])

            if (i != 0) and (i != (n_i - 1)):
            
                if j == 0:

                    df_ij_l = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    df_ij_u_downstream = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j,i-1)) + ".csv")
                    df_ij_u_upstream = get_euclidean_distance(folder_path + str((j,i+1)) + ".csv", folder_path + str((j,i)) + ".csv")
                    dl_ij = np.abs(l[j+1] - l[j])
                    du_ij_downstream = np.abs(u[i] - u[i-1])
                    du_ij_upstream = np.abs(u[i+1]-u[i])

                    gradient_dict[(i,j)] = np.array([df_ij_l / dl_ij, df_ij_u_downstream / (2*du_ij_downstream) + df_ij_u_upstream / (2*du_ij_upstream)])
            
                elif j == (n_j - 1):

                    df_ij_l = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j-1,i)) + ".csv")
                    df_ij_u_downstream = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j,i-1)) + ".csv")
                    df_ij_u_upstream = get_euclidean_distance(folder_path + str((j,i+1)) + ".csv", folder_path + str((j,i)) + ".csv")
                    dl_ij = np.abs(l[j] - l[j-1])
                    du_ij_downstream = np.abs(u[i] - u[i-1])
                    du_ij_upstream = np.abs(u[i+1]-u[i])

                    gradient_dict[(i,j)] = np.array([df_ij_l / dl_ij, df_ij_u_downstream / (2*du_ij_downstream) + df_ij_u_upstream / (2*du_ij_upstream)])

                else:

                    df_ij_l_downstream = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j-1,i)) + ".csv")
                    df_ij_l_upstream = get_euclidean_distance(folder_path + str((j+1,i)) + ".csv", folder_path + str((j,i)) + ".csv")
                    df_ij_u_downstream = get_euclidean_distance(folder_path + str((j,i)) + ".csv", folder_path + str((j,i-1)) + ".csv")
                    df_ij_u_upstream = get_euclidean_distance(folder_path + str((j,i+1)) + ".csv", folder_path + str((j,i)) + ".csv")
                    dl_ij_downstream = np.abs(l[j] - l[j-1])
                    dl_ij_upstream = np.abs(l[j+1]-l[j])
                    du_ij_downstream = np.abs(u[i] - u[i-1])
                    du_ij_upstream = np.abs(u[i+1] - u[i])

                    gradient_dict[(i,j)] = np.array([df_ij_l_downstream / (2*dl_ij_downstream) + df_ij_l_upstream / (2*dl_ij_upstream), df_ij_u_downstream / (2*du_ij_downstream) + df_ij_u_upstream / (2*du_ij_upstream)])

    return gradient_dict, D1_feature_counts, D2_feature_counts



def optimal_threshold(df_dx, features_d1, features_d2, n, delta_ratio_d1, delta_ratio_d2, quantile=False, solver_path=path_to_gurobi):
    """
    Calculates the optimal persistence diagram by solving the following linear program:
    Minimize: W_0*X_0 + ... + W_N*X_N
    Subject to: X_0 + ... + X_N = 1 (Decision variables)
                Fk_0*X_0 + ... + Fk_N*X_N >= delta_k for all k=1,...,(maximum dimension)
    Where: W_i = average weight to neighbors
           Fk_i = number of dimension k features per persistence diagram i
           delta_k = delta_ratio * max(Fk_i)

    When run in a Jupyter notebook, the optimal decision variable is printed in the cell output. The code can be modified to print the decision variable.

    avg_weights = (arr_like) sum(w(i,k)/deg(i)) where k are the neighbors of node i, for all nodes i, must have same order as avg_weights
    features_d1 = (arr_like) feature counts of persistence diagrams, must have same order as avg_weights (D1)
    features_d2 = (arr_like) feature counts of persistence diagrams, must have same order as avg_weights (D2)
    n = number of persistence diagrams
    delta_ratio = (float) the percentage of the max to use as the delta OR percentile (quantile)
    quantile = (Boolean) default False, must be set True if deriving delta_ratio per a quantile
    solver_path = (String) environment variable for desired solver, default = Gurobi
    """

    # Formatting
    if len(features_d1) != len(df_dx) and len(features_d1) != n:
        raise ValueError("Length of dimension one features list must match length of gradients list")
    if len(features_d2) != len(df_dx) and len(features_d2) != n:
        raise ValueError("Length of dimension two features list must match length of gradients list")
    f_d1 = np.asarray(features_d1)
    f_d2 = np.asarray(features_d2)
    w = np.asarray(df_dx)
    
    if quantile:
        delta_d1 = np.percentile(f_d1, delta_ratio_d1)
        delta_d2 = np.percentile(f_d2, delta_ratio_d2)
    else:
        delta_d1 = delta_ratio_d1 * f_d1.max()
        delta_d2 = delta_ratio_d2 * f_d2.max()

    # Configure the solver
    solver = pl.GUROBI(solver_path, msg=0)

    # Create the problem
    prob = pl.LpProblem("Grid Search", pl.LpMinimize)

    # Establish the variables
    x = [pl.LpVariable("Chi_" + str(i),0,1,pl.LpInteger) for i in range(n)]
    x = np.asarray(x)

    # Add the objective function to the problem first
    prob += np.matmul(w.T, x), "Average Distance To Neighbors"

    # Add the constraints second to the objective
    prob += np.sum(x) == 1, "Single Decision Requirement"
    prob += np.matmul(f_d1.T, x) >= delta_d1, "Delta Requirement (D1)"
    prob += np.matmul(f_d2.T, x) >= delta_d2, "Delta Requirement (D2)"

    # Solve (indicate the solver)
    prob.solve(solver)
    # print("Status:", pl.LpStatus[prob.status]) # uncomment to print status
    for v in prob.variables():
        if v.varValue == 1.0:
            # print("Optimal Index: " + v.name, "=", v.varValue) # The nonzero decison variable indicates the index for the PD with the optimal threshold
            return v.name
        


def extract_numbers(text):
  """
  Extract numbers from a string. Function taken from Google AI response

  text: (String) text from which to extract a number
  """
  return re.findall(r'\d+', text)



def get_indices(num, upper_list, lower_list):
    """
    Find the indices (on the 'grid' of networks) corresponding to a given decision variable

    num = (String) a decision variable given in the form of a string (e.g. "Chi_10")
    """
    num = int(extract_numbers(num)[0])
    ct = 0
    for i in range(len(upper_list)):
        for j in range(1,len(lower_list)):
            if ct == num:
                return([i,j])
            ct += 1
    return